In [3]:
import pyspark 
from pyspark import SparkContext, SparkConf


In [4]:
sc = SparkContext("local","Basic RDD Transformation and Actions")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/11 15:23:40 WARN Utils: Your hostname, kingstein, resolves to a loopback address: 127.0.1.1; using 10.96.200.224 instead (on interface wlp3s0)
25/08/11 15:23:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/11 15:23:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
data = ["Spark is Fast","SPark is powerful","SPark is easy to use"]
rdd = sc.parallelize(data)

In [8]:
words = rdd.flatMap(lambda line:line.split(" "))
word_pairs = words.map(lambda word:(word,1))
word_count = word_pairs.reduceByKey(lambda a,b:a+b)
filtered_words = word_count.filter(lambda x:x[1]>1)


In [10]:
print("\n -- Retrival Action ---")
print("Collect : ",words.collect())
print("Take (5):",words.take(5))
print("Take Sample (no replacement,4):",words.takeSample(False,4))
print("Take Ordered (alphabetical,5):",words.takeOrdered(5))
print("Top 5 : ",words.top(5))
print("First Element : ",words.first())
print("IS empty :",words.isEmpty())
print("\n Foreach output")
words.foreach(lambda w: print("Word:",w))


 -- Retrival Action ---
Collect :  ['Spark', 'is', 'Fast', 'SPark', 'is', 'powerful', 'SPark', 'is', 'easy', 'to', 'use']
Take (5): ['Spark', 'is', 'Fast', 'SPark', 'is']
Take Sample (no replacement,4): ['is', 'Fast', 'powerful', 'SPark']
Take Ordered (alphabetical,5): ['Fast', 'SPark', 'SPark', 'Spark', 'easy']
Top 5 :  ['use', 'to', 'powerful', 'is', 'is']
First Element :  Spark
IS empty : False

 Foreach output


Word: Spark
Word: is
Word: Fast
Word: SPark
Word: is
Word: powerful
Word: SPark
Word: is
Word: easy
Word: to
Word: use


In [11]:
print("\n ---Aggregation Actions ---")
print("Count",words.count())
print("CountByValue",dict(words.countByValue()))
print("Reduce (total word count):",words.map(lambda x:1).reduce(lambda a,b: a+b))
print("Fold (total word count):", words.map(lambda x:1).fold(0,lambda a,b: a+b))


 ---Aggregation Actions ---
Count 11
CountByValue {'Spark': 1, 'is': 3, 'Fast': 1, 'SPark': 2, 'powerful': 1, 'easy': 1, 'to': 1, 'use': 1}
Reduce (total word count): 11
Fold (total word count): 11


In [13]:
agg_result = words.aggregate(
(0, 0), # (total length, word count)
lambda acc, value: (acc[0] + len(value), acc[1] + 1), # seqOp
lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1]) # combOp
)

In [14]:
avg_word_length = agg_result[0]/agg_result[1]
print("Aggregate (avg word length):",avg_word_length)

Aggregate (avg word length): 3.8181818181818183


In [15]:
sc.stop()